In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


import tensorflow as tf
from keras.models import Sequential,load_model
from keras import layers,Input
from keras.losses import binary_crossentropy
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras.regularizers import L1L2
import keras.backend as K

2023-10-12 16:25:17.539888: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-12 16:25:18.654374: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.__version__)

Num GPUs Available:  1
2.13.1


In [ ]:
arry=np.array([[[1,11],[2,12],[3,13],[4,14],[5,15],[6,16],[7,17],[8,18],[9,19],[10,20]],[[1,11],[2,12],[3,13],[4,14],[5,15],[6,16],[7,17],[8,18],[9,19],[10,20]]])
# arry=np.array([[[1,2,3,4,5,6,7,8,9,10],[11,12,13,14,15,16,17,18,19,20]],[[1,2,3,4,5,6,7,8,9,10],[11,12,13,14,15,16,17,18,19,20]]])
print(arry.shape)
print(arry)
print(arry.reshape((2,5,4)))

In [3]:
state0=np.load('../Data/new/2gsps/state0.npy')
state1=np.load('../Data/new/2gsps/state1.npy')
output0=np.zeros((state0.shape[0]))
output1=np.ones((state1.shape[0]))
print(state0.shape)

(6000, 8192, 2)


In [4]:
x=np.vstack((state0,state1))
print(x.shape)
y=np.hstack((output0,output1))
print(y.shape)

(12000, 8192, 2)
(12000,)


In [ ]:

st,ed=700,2700
x_train, x_test, y_train, y_test = train_test_split(x[:,st:ed,:], y, test_size=0.30, random_state=45)
print(x_train.shape)
print(y_train.sum())

x_train=np.mean(x_train,axis=1)
x_test=np.mean(x_test,axis=1)

# x_train=x_train.reshape((x_train.shape[0],x_train.shape[1]*2))
# x_test=x_test.reshape((x_test.shape[0],x_test.shape[1]*2))
print(x_train.shape)

In [ ]:
2**16

## Logistic Regression

In [ ]:
# x_trainC=x_train.reshape((x_train.shape[0],x_train.shape[1]*2))
# x_testC=x_test.reshape((x_test.shape[0],x_test.shape[1]*2))
classifier = LogisticRegression(random_state = 42, max_iter=500)
classifier.fit(x_train, y_train)
print(classifier.score(x_test,y_test))
pred = classifier.predict(x_test)

## SVM

In [ ]:
model=SVC(decision_function_shape='ovo')
model.fit(x_train,y_train)
print(model.score(x_test,y_test))

## GMM

In [ ]:
Y=list(zip(np.append(np.mean(state0[:,st:ed,0],axis=1),np.mean(state1[:,st:ed,0],axis=1)),np.append(np.mean(state0[:,st:ed,1],axis=1),np.mean(state1[:,st:ed,1],axis=1))))
gmm_Y=GaussianMixture(n_components=2,covariance_type='full').fit(Y)
bitstring=np.split(gmm_Y.predict(Y),2)
print('P(0|0)=',len(bitstring[0][bitstring[0]==0])/len(bitstring[0]))
print('P(1|1)=',len(bitstring[1][bitstring[1]==1])/len(bitstring[1]))
X=list(zip(np.mean(state0[:,st:ed,0],axis=1),np.mean(state0[:,st:ed,1],axis=1)))
gmm_Y_1=GaussianMixture(n_components=1,covariance_type='spherical').fit(X)
X=list(zip(np.mean(state1[:,st:ed,0],axis=1),np.mean(state1[:,st:ed,1],axis=1)))
gmm_Y_2=GaussianMixture(n_components=1,covariance_type='spherical').fit(X)

## NN

In [ ]:
def arch():
    model=Sequential()
    model.add(Input(shape=(2)))
    model.add(Dense(32,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(16,activation='relu'))
    model.add(Dropout(0.5))
    # model.add(Dense(16,activation='relu'))
    # # model.add(Dropout(0.8))
    # model.add(Dense(16,activation='relu'))
    # # model.add(Dropout(0.8))
    # model.add(Dense(32,activation='relu'))
    # # model.add(Dropout(0.8))
    # model.add(Dense(16,activation='relu'))
    # # model.add(Dropout(0.8))
    # model.add(Dense(4,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    return model
arch().summary()

In [ ]:
def bceloss(y_true, y_pred):
    # Clip the prediction value to prevent log(0) error
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # Calculate the weighted binary cross entropy loss
    loss = -(0.1 * (y_true * K.log(y_pred)) + 0.9*((1 - y_true) * K.log(1 - y_pred)))
    return K.mean(loss, axis=-1)

model=arch()
model.summary
opt=SGD( learning_rate=0.01, momentum=0.9)
model.compile(loss=binary_crossentropy, optimizer='adam', metrics=['accuracy'])
model_path="../Model/NN2/{epoch:02d}-{val_accuracy:.4f}.h5"
checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
H=model.fit(x_train,y_train,
          validation_data=(x_test, y_test),
          epochs=100,batch_size=16,
          callbacks=callbacks_list,
          verbose=1)

In [ ]:
count=0
count1=0
y_pred=model.predict(x_test)
for i in range(y_pred.shape[0]):
    if y_test[i]==0:
        count+=1
        if y_pred[i,0]<0.5:
            count1+=1
print(count,count1)

## GDA

In [ ]:
import numpy as np
from scipy.stats import multivariate_normal

class GaussianDiscriminantAnalysis:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.class_priors = np.array([np.mean(y == c) for c in self.classes])

        self.means = []
        self.cov_matrices = []
        for c in self.classes:
            X_c = X[y == c]
            mean_c = np.mean(X_c, axis=0)
            cov_matrix_c = np.cov(X_c, rowvar=False)
            self.means.append(mean_c)
            self.cov_matrices.append(cov_matrix_c)

    def predict(self, X):
        posteriors = []

        for c in self.classes:
            class_prior = self.class_priors[int(c)]
            mean = self.means[int(c)]
            cov_matrix = self.cov_matrices[int(c)]
            mvn = multivariate_normal(mean=mean, cov=cov_matrix)
            posterior = class_prior * mvn.pdf(X)
            posteriors.append(posterior)

        posteriors = np.array(posteriors).T
        predicted_labels = np.argmax(posteriors, axis=1)

        return predicted_labels



# Initialize and train GDA
gda = GaussianDiscriminantAnalysis()
gda.fit(x_train, y_train)

# Make predictions
y_pred = gda.predict(x_train)
# Calculate accuracy
accuracy = np.mean(y_pred == y_train)
print(f"Accuracy: {accuracy * 100:.2f}%")

## RNN

In [18]:
min,max=-(2**16),(2**16)
print('Min: %.3f, Max: %.3f' % (x.min(), x.max()))
x= (x-min)/(max-min)
print('Min: %.3f, Max: %.3f' % (x.min(), x.max()))

Min: 0.254, Max: 0.718
Min: 0.500, Max: 0.500


In [24]:
st,ed=1400,4400
x_train, x_test, y_train, y_test = train_test_split(x[:,st:ed,:], y, test_size=0.30, random_state=45)
x_train=x_train.reshape((x_train.shape[0],6000))
x_test=x_test.reshape((x_test.shape[0],6000))
print(x_train.shape)
print(y_train.sum())

(8400, 6000)
4233.0


In [33]:
def rnn():
    model=Sequential()
    model.add(Input(shape=(200,20)))
    model.add(layers.LSTM(units=16, return_sequences=True))
    model.add(layers.LSTM(units=8, return_sequences=True))
    model.add(layers.LSTM(units=4, return_sequences=False))
#     model.add(layers.Dense(4,activation='relu'))
    model.add(layers.Dense(1,activation='sigmoid'))
    return model

def nn():
    model=Sequential()
    model.add(Input(shape=(6000)))
    model.add(layers.Dense(8192,activation='relu'))
    model.add(layers.Dense(4096,activation='relu'))
    model.add(layers.Dense(2048,activation='relu'))
    model.add(layers.Dense(1024,activation='relu'))
    model.add(layers.Dense(512,activation='relu'))
    model.add(layers.Dense(256,activation='relu'))
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(32,activation='relu'))
    model.add(layers.Dense(8,activation='relu'))
    model.add(layers.Dense(2,activation='relu'))
    model.add(layers.Dense(1,activation='sigmoid'))
    return model

In [34]:
model=rnn()
model.summary()
opt=SGD( learning_rate=0.01, momentum=0.9)
model.compile(loss=binary_crossentropy, optimizer=opt, metrics=['accuracy'])
model_path="../Model/RNN_new/{epoch:02d}-{val_accuracy:.4f}.h5"
checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_41 (Dense)            (None, 8192)              49160192  
                                                                 
 dense_42 (Dense)            (None, 4096)              33558528  
                                                                 
 dense_43 (Dense)            (None, 2048)              8390656   
                                                                 
 dense_44 (Dense)            (None, 1024)              2098176   
                                                                 
 dense_45 (Dense)            (None, 512)               524800    
                                                                 
 dense_46 (Dense)            (None, 256)               131328    
                                                                 
 dense_47 (Dense)            (None, 64)               

In [35]:
with tf.device("/gpu:0"):
    H=model.fit(x_train,y_train,
            validation_data=(x_test, y_test),
            epochs=200,batch_size=256,
            callbacks=callbacks_list,
            verbose=1)

Epoch 1/200
32/33 [============================>.] - ETA: 0s - loss: 0.6932 - accuracy: 0.4957
Epoch 1: val_accuracy improved from -inf to 0.49083, saving model to ../Model/RNN_new/01-0.4908.h5
33/33 [==============================] - 4s 74ms/step - loss: 0.6932 - accuracy: 0.4958 - val_loss: 0.6932 - val_accuracy: 0.4908
Epoch 2/200
29/33 [=========================>....] - ETA: 0s - loss: 0.6931 - accuracy: 0.5047
Epoch 2: val_accuracy did not improve from 0.49083
33/33 [==============================] - 0s 11ms/step - loss: 0.6931 - accuracy: 0.5039 - val_loss: 0.6933 - val_accuracy: 0.4908
Epoch 3/200
28/33 [========================>.....] - ETA: 0s - loss: 0.6931 - accuracy: 0.5054
Epoch 3: val_accuracy did not improve from 0.49083
33/33 [==============================] - 0s 12ms/step - loss: 0.6932 - accuracy: 0.5039 - val_loss: 0.6933 - val_accuracy: 0.4908
Epoch 4/200
33/33 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5039
Epoch 4: val_accuracy did not

33/33 [==============================] - 0s 11ms/step - loss: 0.6932 - accuracy: 0.5039 - val_loss: 0.6933 - val_accuracy: 0.4908
Epoch 31/200
30/33 [==========================>...] - ETA: 0s - loss: 0.6931 - accuracy: 0.5046
Epoch 31: val_accuracy did not improve from 0.49083
33/33 [==============================] - 0s 11ms/step - loss: 0.6931 - accuracy: 0.5039 - val_loss: 0.6933 - val_accuracy: 0.4908
Epoch 32/200
29/33 [=========================>....] - ETA: 0s - loss: 0.6931 - accuracy: 0.5061
Epoch 32: val_accuracy did not improve from 0.49083
33/33 [==============================] - 0s 12ms/step - loss: 0.6931 - accuracy: 0.5039 - val_loss: 0.6933 - val_accuracy: 0.4908
Epoch 33/200
28/33 [========================>.....] - ETA: 0s - loss: 0.6932 - accuracy: 0.5013
Epoch 33: val_accuracy did not improve from 0.49083
33/33 [==============================] - 0s 13ms/step - loss: 0.6932 - accuracy: 0.5039 - val_loss: 0.6933 - val_accuracy: 0.4908
Epoch 34/200
29/33 [================

29/33 [=========================>....] - ETA: 0s - loss: 0.6932 - accuracy: 0.5016
Epoch 60: val_accuracy did not improve from 0.49083
33/33 [==============================] - 0s 11ms/step - loss: 0.6931 - accuracy: 0.5039 - val_loss: 0.6933 - val_accuracy: 0.4908
Epoch 61/200
29/33 [=========================>....] - ETA: 0s - loss: 0.6932 - accuracy: 0.5028
Epoch 61: val_accuracy did not improve from 0.49083
33/33 [==============================] - 0s 11ms/step - loss: 0.6932 - accuracy: 0.5039 - val_loss: 0.6933 - val_accuracy: 0.4908
Epoch 62/200
30/33 [==========================>...] - ETA: 0s - loss: 0.6931 - accuracy: 0.5066
Epoch 62: val_accuracy did not improve from 0.49083
33/33 [==============================] - 0s 12ms/step - loss: 0.6931 - accuracy: 0.5039 - val_loss: 0.6935 - val_accuracy: 0.4908
Epoch 63/200
29/33 [=========================>....] - ETA: 0s - loss: 0.6932 - accuracy: 0.5027
Epoch 63: val_accuracy did not improve from 0.49083
33/33 [========================

33/33 [==============================] - 0s 11ms/step - loss: 0.6931 - accuracy: 0.5039 - val_loss: 0.6933 - val_accuracy: 0.4908
Epoch 90/200
33/33 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.5039
Epoch 90: val_accuracy did not improve from 0.49083
33/33 [==============================] - 0s 10ms/step - loss: 0.6932 - accuracy: 0.5039 - val_loss: 0.6934 - val_accuracy: 0.4908
Epoch 91/200
33/33 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5039
Epoch 91: val_accuracy did not improve from 0.49083
33/33 [==============================] - 0s 11ms/step - loss: 0.6931 - accuracy: 0.5039 - val_loss: 0.6933 - val_accuracy: 0.4908
Epoch 92/200
26/33 [======================>.......] - ETA: 0s - loss: 0.6931 - accuracy: 0.5059
Epoch 92: val_accuracy did not improve from 0.49083
33/33 [==============================] - 0s 10ms/step - loss: 0.6931 - accuracy: 0.5039 - val_loss: 0.6933 - val_accuracy: 0.4908
Epoch 93/200
31/33 [================

Epoch 119/200
29/33 [=========================>....] - ETA: 0s - loss: 0.6931 - accuracy: 0.5054
Epoch 119: val_accuracy did not improve from 0.49083
33/33 [==============================] - 0s 12ms/step - loss: 0.6931 - accuracy: 0.5039 - val_loss: 0.6933 - val_accuracy: 0.4908
Epoch 120/200
32/33 [============================>.] - ETA: 0s - loss: 0.6931 - accuracy: 0.5032
Epoch 120: val_accuracy did not improve from 0.49083
33/33 [==============================] - 0s 11ms/step - loss: 0.6931 - accuracy: 0.5039 - val_loss: 0.6933 - val_accuracy: 0.4908
Epoch 121/200
31/33 [===========================>..] - ETA: 0s - loss: 0.6931 - accuracy: 0.5025
Epoch 121: val_accuracy did not improve from 0.49083
33/33 [==============================] - 0s 11ms/step - loss: 0.6931 - accuracy: 0.5039 - val_loss: 0.6933 - val_accuracy: 0.4908
Epoch 122/200
30/33 [==========================>...] - ETA: 0s - loss: 0.6932 - accuracy: 0.5039
Epoch 122: val_accuracy did not improve from 0.49083
33/33 [===

33/33 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5039
Epoch 148: val_accuracy did not improve from 0.49083
33/33 [==============================] - 0s 11ms/step - loss: 0.6931 - accuracy: 0.5039 - val_loss: 0.6933 - val_accuracy: 0.4908
Epoch 149/200
32/33 [============================>.] - ETA: 0s - loss: 0.6931 - accuracy: 0.5051
Epoch 149: val_accuracy did not improve from 0.49083
33/33 [==============================] - 0s 11ms/step - loss: 0.6931 - accuracy: 0.5039 - val_loss: 0.6933 - val_accuracy: 0.4908
Epoch 150/200
25/33 [=====================>........] - ETA: 0s - loss: 0.6931 - accuracy: 0.5058
Epoch 150: val_accuracy did not improve from 0.49083
33/33 [==============================] - 0s 10ms/step - loss: 0.6931 - accuracy: 0.5039 - val_loss: 0.6934 - val_accuracy: 0.4908
Epoch 151/200
33/33 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5039
Epoch 151: val_accuracy did not improve from 0.49083
33/33 [=================

KeyboardInterrupt: 

## XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
model = xgb.XGBClassifier(
    learning_rate=0.01,
    n_estimators=100,
    max_depth=9,
    objective='binary:logistic',
    eval_metric='logloss'
)

In [ ]:
model.fit(x_train, y_train)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

In [ ]:
print(accuracy)